In [1]:
import torch
import torch.nn as nn
import time 
from time import sleep
import datetime
import os 
import cv2
import numpy as np 
import torch.optim as optim
import matplotlib.pyplot as plt 
from torch.utils.data import DataLoader 
from sklearn.model_selection import train_test_split
import pandas as pd
import TrainHelper as th
import CustomModels as cm
import CustomImageDataset as cid
import torch.nn.functional as F
import LossFunction as lf 
import sys
import glob 
from torchvision.transforms.v2 import GaussianBlur
from torchvision.io import read_image, ImageReadMode
device = ("cuda" if torch.cuda.is_available() else "cpu")

sys.path.append(os.path.join('..','segment-utils'))
import VideoAnnotationHelper as vah

!pip install torchinfo
from torchinfo import summary

C:\Users\bajajp\AppData\Local\anaconda3\envs\Xmen\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\bajajp\AppData\Local\anaconda3\envs\Xmen\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please

In [2]:
noHub_dir = "C:/Lumena/train_data/NH/"
images1 = glob.glob(noHub_dir + "images_cropped_256x256/*.png")
labels1 = []
label_paths = glob.glob(noHub_dir + "masksLined_cropped_256x256/*.png")
for k in range(len(label_paths)):
    ele = []
    ele.append(label_paths[k])
    ele.append((-1, -1,-1,-1,0))
    labels1.append(ele)

In [3]:
noHub_dir = "C:/Lumena/train_data/NH2/"
images2 = glob.glob(noHub_dir + "images_cropped_256x256/*.png")
labels2 = []
label_paths = glob.glob(noHub_dir + "masksLined_cropped_256x256/*.png")
for k in range(len(label_paths)):
    ele = []
    ele.append(label_paths[k])
    ele.append((-1, -1,-1,-1,0))
    labels2.append(ele)

In [4]:
fineTune = True  
model_path = "//cgmqnap.clearguide.local/data/Needles/Lumena/models/hub_regression/model_cyc_1_ep_400_bs_32_lr_0.0001_20231228_T171234.pt"

parent_dir = "C:/Lumena"
train_data_folder = "train_data"
sub_folders = ["EL", "PC2", "PR", "EL_2", "CNMC", "CNMC.2023.11.07", "SS"]

batch_size = 128
epochs = 200
cycles = 1 

encoder_lr = 1e-4  
regA_lr = 1e-4
regB_lr = 1e-6

stop_encoder_requires_grad = True    
stop_regA_requires_grad = True  
stop_regB_requires_grad = False  

FullRange = False 
Gray = False
ifTrain = True 
AddMskCha = True 

observe_path="D:/test_dir"

old_model_path = "//cgmqnap.clearguide.local/data/Needles/Lumena/models/shaft_segmentation_models/3ChaInpute_RegARegBCombined_20231116_T124859.pt"

In [5]:
csv_filename = ["cropped_256x256.csv"]
img_dir = ["images_cropped_256x256"]
mask_dir = ["masksLined_cropped_256x256"]

images3 = []
labels3 = []

for k in range(len(csv_filename)):
    dfs = []
    for ele in sub_folders:
        df = pd.read_csv(parent_dir + "/" + train_data_folder + "/" + ele +"/" + csv_filename[k])
        dfs.append(df)

    for i in range(len(dfs)): 
        for name in zip(dfs[i]["Images"]):  
            label_ele = [] 
            images3.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + img_dir[k] + "/" + name[0])
            label_ele.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + mask_dir[k] + "/" + name[0])
            label_ele.append((-1,-1,-1,-1,1))
            labels3.append(label_ele)


In [6]:
csv_filename = ["no_hub_cropped_256x256.csv"]
img_dir = ["images_no_hub_cropped_256x256"]
mask_dir = ["masksLined_no_hub_cropped_256x256"]

images4 = []
labels4 = []

for k in range(len(csv_filename)):
    dfs = []
    for ele in sub_folders:
        df = pd.read_csv(parent_dir + "/" + train_data_folder + "/" + ele +"/" + csv_filename[k])
        dfs.append(df)

    for i in range(len(dfs)): 
        for name in zip(dfs[i]["Images"]): 
            label_ele = [] 
            images4.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + img_dir[k] + "/" + name[0])
            label_ele.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + mask_dir[k] + "/" + name[0])
            label_ele.append((-1,-1,-1,-1,0))
            labels4.append(label_ele)

In [7]:
images4, _, labels4, _ = train_test_split(images4, labels4, test_size = 0.1)

In [8]:
images_path = images1 + images2 + images3 + images4
labels = labels1 + labels2 + labels3 + labels4

In [9]:
X_train, X_val, y_train, y_val = train_test_split(images_path, labels, test_size = 0.2)

train_dataset = cid.Custom_ImageRegressor_Dataset2(X_train, y_train, ifgray = Gray, ifTrain = ifTrain, ifFullRange = FullRange, ifAddMskCha = AddMskCha)
val_dataset = cid.Custom_ImageRegressor_Dataset2(X_val, y_val, ifgray = Gray, ifTrain = ifTrain, ifFullRange = FullRange, ifAddMskCha = AddMskCha)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True,num_workers=4,pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False,num_workers=4,pin_memory = True)

In [10]:
model_name = "model" 
model_name += "_cyc_" + str(cycles) + "_ep_" + str(epochs) + "_bs_" + str(batch_size)
model_name += "_lr_" + str(regA_lr)
model_folder = model_name +  "_{:%Y%m%d_T%H%M%S}".format(datetime.datetime.now())
 
vah.createFolder(observe_path + "/"  + model_folder)

Created folder 'D:/test_dir/model_cyc_1_ep_200_bs_128_lr_0.0001_20240105_T114613'.


In [11]:
if fineTune: 
    print("Model loaded for finetune")
    model = torch.jit.load(model_path)

Model loaded for finetune


In [12]:
i = 0
for name, param in model.named_parameters():
    if param.requires_grad == True:
        print("NO.", i, name)
        i += 1

NO. 0 conv1.0.conv_block.0.conv_block.0.weight
NO. 1 conv1.0.conv_block.0.conv_block.0.bias
NO. 2 conv1.0.conv_block.0.conv_block.1.weight
NO. 3 conv1.0.conv_block.0.conv_block.1.bias
NO. 4 conv1.1.conv_block.0.conv_block.0.weight
NO. 5 conv1.1.conv_block.0.conv_block.0.bias
NO. 6 conv1.1.conv_block.0.conv_block.1.weight
NO. 7 conv1.1.conv_block.0.conv_block.1.bias
NO. 8 conv2.0.conv_block.0.conv_block.0.weight
NO. 9 conv2.0.conv_block.0.conv_block.0.bias
NO. 10 conv2.0.conv_block.0.conv_block.1.weight
NO. 11 conv2.0.conv_block.0.conv_block.1.bias
NO. 12 conv2.1.conv_block.0.conv_block.0.weight
NO. 13 conv2.1.conv_block.0.conv_block.0.bias
NO. 14 conv2.1.conv_block.0.conv_block.1.weight
NO. 15 conv2.1.conv_block.0.conv_block.1.bias
NO. 16 conv3.0.layer.0.group1.conv_block.0.weight
NO. 17 conv3.0.layer.0.group1.conv_block.0.bias
NO. 18 conv3.0.layer.0.group1.conv_block.1.weight
NO. 19 conv3.0.layer.0.group1.conv_block.1.bias
NO. 20 conv3.0.layer.0.group2.0.conv_block.0.conv_block.0.weig

In [13]:
 i = 0 
encoder_list = []
regressorA_list = []
regressorB_list = [] 

for param in model.parameters(): 
    if i < 112: 
        if stop_encoder_requires_grad:
            param.requires_grad = False 
        encoder_list.append(param)
    elif i < 140: 
        if stop_regA_requires_grad: 
            param.requires_grad = False 
        regressorA_list.append(param)
    else: 
        if stop_regB_requires_grad: 
            param.requires_grad = False 
        regressorB_list.append(param)
        
    i += 1

In [14]:
for name, param in model.named_parameters():
    if param.requires_grad == True:
        print(name)

fcb.0.weight
fcb.0.bias
fcb.1.weight
fcb.1.bias
fcb.3.weight
fcb.3.bias
fcb.4.weight
fcb.4.bias
fcb.6.weight
fcb.6.bias
fcb.7.weight
fcb.7.bias
fcb.9.weight
fcb.9.bias


In [15]:
 lossFun1 = nn.BCELoss()

In [16]:
 regB_lr = 1e-4

In [17]:
for j in range(cycles):

    print("Training Cycle: ", j+1)

    optimizer = torch.optim.Adam([#{"params":encoder_list, "lr":encoder_lr},
                                 {"params":regressorB_list, "lr":regB_lr},])
    
    for epoch in range(epochs): 
        
        train_loss = 0.0
        val_loss = 0.0        
        start_time = time.time() 
        
        model.train()
        for data in train_loader:
            images, true_yx, true_p= data 
            timex = time.process_time()
            images = images.to(device)
            true_p = true_p.to(device)
            optimizer.zero_grad()
            
            pred_yx, pred_p = model(images)

            loss1 = lossFun1.forward(pred_p.float(),true_p.float())
            
            #print(loss1.item())
            
            loss1.backward()
            optimizer.step()
            train_loss += loss1

            
        model.eval()    
        for data in val_loader: 
            
            images, true_yx, true_p= data
            images = images.to(device)
            true_p = true_p.to(device)
            
            with torch.no_grad(): 
                pred_yx, pred_p = model(images)
                loss1 = lossFun1.forward(pred_p.float(),true_p.float())
                
            val_loss += loss1

        
        train_loss = train_loss/len(train_loader)    
        val_loss = val_loss/len(val_loader)
        
        
        print('Epoch: {}  \tTraining Loss: {:.6f} \t Vali Loss: {:.6f} \t Exe Time: {:.6f}sec'.format(
        epoch, 
        train_loss, 
        val_loss,
        time.time() - start_time))
        
        if epoch % 10 == 0: 
 
            model_script = torch.jit.script(model)
            temp_name = model_name + "_{:%Y%m%d_T%H%M%S}".format(datetime.datetime.now()) + ".pt"
            model_script.save(observe_path + "/"  + model_folder + "/" + temp_name)
            print("Temprary model saved: ", temp_name) 

Training Cycle:  1
Epoch: 0  	Training Loss: 0.358357 	 Vali Loss: 0.278641 	 Exe Time: 25.254119sec
Temprary model saved:  model_cyc_1_ep_200_bs_128_lr_0.0001_20240105_T114641.pt
Epoch: 1  	Training Loss: 0.292989 	 Vali Loss: 0.268150 	 Exe Time: 15.518880sec
Epoch: 2  	Training Loss: 0.279812 	 Vali Loss: 0.270083 	 Exe Time: 15.512229sec
Epoch: 3  	Training Loss: 0.270842 	 Vali Loss: 0.263631 	 Exe Time: 15.316238sec
Epoch: 4  	Training Loss: 0.259621 	 Vali Loss: 0.257873 	 Exe Time: 15.300140sec
Epoch: 5  	Training Loss: 0.250651 	 Vali Loss: 0.248484 	 Exe Time: 15.218940sec
Epoch: 6  	Training Loss: 0.241521 	 Vali Loss: 0.241823 	 Exe Time: 15.394868sec
Epoch: 7  	Training Loss: 0.234424 	 Vali Loss: 0.238206 	 Exe Time: 15.279839sec
Epoch: 8  	Training Loss: 0.226636 	 Vali Loss: 0.230219 	 Exe Time: 15.294105sec
Epoch: 9  	Training Loss: 0.219226 	 Vali Loss: 0.224520 	 Exe Time: 15.183533sec
Epoch: 10  	Training Loss: 0.211877 	 Vali Loss: 0.216498 	 Exe Time: 15.392917sec

KeyboardInterrupt: 